In [1]:
import os
import time
from copy import copy
from pathlib import Path
from joblib import Memory
from scipy import sparse
from scipy.sparse import csc_matrix
from scipy.special import expit
from scipy.stats import pearsonr

from sklearn import svm
from sklearn import tree
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample, shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, normalize

from xgboost import XGBClassifier
from category_encoders import TargetEncoder

import pyarrow.feather as feather
import matplotlib.pyplot as plt
import plotnine as p9
import pandas as pd
import numpy as np

rand_state = 5991

In [2]:
# FUNCTIONS
def split_data(xx, yy, testsize = 1000):
    xtrain, xtest, y_train, y_test = train_test_split(xx,
                                                      yy,
                                                      test_size = testsize,
                                                      random_state = rand_state)
    xtrain, xvalid, y_train, y_valid = train_test_split(xtrain, y_train, 
                                                        test_size = testsize,
                                                        random_state = rand_state)
    y_train = np.array(y_train).reshape(-1, 1)
    y_test = np.array(y_test).reshape(-1, 1)
    y_valid = np.array(y_valid).reshape(-1, 1)

    print(" SHAPE of xtrain:", xtrain.shape)
    print("SHAPE of y_train:", y_train.shape)
    print("  SHAPE of xtest:", xtest.shape)
    print(" SHAPE of y_test:", y_test.shape)
    print(" SHAPE of xvalid:", xvalid.shape)
    print("SHAPE of y_valid:", y_valid.shape)

    return xtrain, y_train, xtest, y_test, xvalid, y_valid 

def get_acc_auc(y, p):
    acc = np.sum(y == p) / len(y)
    auc = roc_auc_score(y, p)
    return acc, auc



In [14]:
# Paths
idcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/idcsc.feather"
cipcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/cip_csc.feather"
geoloccsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/geolocation_csc.feather"
instdemocsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/inst_demographic_csc.feather"
studdemocsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/stud_demographic_csc.feather"
pcipcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/pcip_csc.feather"
numcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/num_csc.feather"
tarcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/tarcsc.feather"
fulldf = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/final_df/full_df.feather"

In [15]:
id_csc = feather.read_feather(idcsc)
cip_csc = feather.read_feather(cipcsc)
geolocation_csc = feather.read_feather(geoloccsc)
inst_demographic_csc = feather.read_feather(instdemocsc)
stud_demographic_csc = feather.read_feather(studdemocsc)
pcip_csc = feather.read_feather(pcipcsc)
num_csc = feather.read_feather(numcsc)
tar_csc = feather.read_feather(tarcsc)

full_df = feather.read_feather(fulldf)

missing = pd.DataFrame(full_df.isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] > 0]

,index,0


In [19]:
# Do the ids column in the target frame match the the ids column in the full frame for each unique UNITID?
# df = pd.merge(tar_csc[["ids", "UNITID"]],
#          full_df[["ids", "UNITID", "INSTNM"]],
#          how = "left", left_on = "UNITID", right_on = "UNITID",
#          suffixes=("_target", "_full"))
# df["diff"] = df["ids_target"] - df["ids_full"]
# df.loc[df["diff"] > 0 ]
# # Yes

In [17]:
ys = tar_csc["PELLCAT"].copy()
xs = full_df.drop(["ids", "UNITID"], axis = 1).copy()
print("SHAPE of xs:", xs.shape)
print("SHAPE of ys:", ys.shape)
xtrain, y_train, xtest, y_test, xvalid, y_valid = split_data(xs, ys, testsize = 1000)

SHAPE of xs: (5879, 318)
SHAPE of ys: (5879,)
 SHAPE of xtrain: (3879, 318)
SHAPE of y_train: (3879, 1)
  SHAPE of xtest: (1000, 318)
 SHAPE of y_test: (1000, 1)
 SHAPE of xvalid: (1000, 318)
SHAPE of y_valid: (1000, 1)


In [ ]:
# drp = ["ST_FIPS", "MENONLY", "WOMENONLY", "TRIBAL", "PBI", "NANTI", "ANNHI", "ANNHI_0.0", "HBCU", 
#        "PBI_0.0", "TRIBAL_0.0", "HSI", "HBCU_0.0", "ANNHI", "HBCU", "HSI", "NANTI_0.0", "HSI_0.0",
#        "MAIN_0", "MAIN", "HCM2", "AANAPII", "AANAPII_0.0", "CURROPER", "CURROPER_0", "ZIP2", "ZIP4",
#        "ST_FIPS","ACCRED_NONE","OPENADMP","HIGHDEG","PREDDEG","ICLEVEL","CONTROL","OPEFLAG","BBRR2_FED_UG_N",
#        "ZIP5","BBRR2_FED_UGNOCOMP_N","LPSTAFFORD_CNT", "BBRR2_PP_UG_N","LPPPLUS_CNT"]

# xtrain = xtrain.drop(drp, axis = 1)
# xvalid = xvalid.drop(drp, axis = 1)
# xtest = xtest.drop(drp, axis = 1)

### Checking for non-linear associations between individual features and the binary target variable

In [20]:
print(xtrain.shape)

xnot = list(xtrain.columns[xtrain.columns.str.endswith("ASSOC") | xtrain.columns.str.endswith("BACHL") |
                      xtrain.columns.str.endswith("CERT1") | xtrain.columns.str.endswith("CERT2") |
                      xtrain.columns.str.endswith("CERT4") | xtrain.columns.str.startswith("DBRR")])
xnot2 = ["INSTNM", "ACCREDAGENCY", "ACCREDCODE", "CITY", "Season", "STABBR", "T4APPROVALDATE", "ZIP", "LATITUDE", "LONGITUDE"]
xnot.extend(xnot2)


print("variables to be removed:")
print(len(xnot))
print("remaining variables:")

features = xtrain.columns.drop(xnot)

print(len(features))

missing = pd.DataFrame(xtrain[features].isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] > 0]

(3879, 318)
variables to be removed:
86
remaining variables:


KeyError: "['INSTNM' 'ACCREDAGENCY' 'ACCREDCODE' 'CITY' 'Season' 'STABBR'\n 'T4APPROVALDATE' 'ZIP' 'LATITUDE' 'LONGITUDE'] not found in axis"

In [22]:
scaler = StandardScaler()
poly = PolynomialFeatures(3)

scaler.fit(xtrain[features])
xtrain = pd.DataFrame(scaler.transform(xtrain[features]), columns = features)
xvalid = pd.DataFrame(scaler.transform(xvalid[features]), columns = features)
xtest = pd.DataFrame(scaler.transform(xtest[features]), columns = features)

log_reg = LogisticRegression(solver = "saga",
                             random_state = rand_state,
                             penalty = "l1",
                             class_weight = "balanced",
                             max_iter = 1000)

print("xtrain before: ", xtrain.shape)

xtrain before:  (3879, 237)


In [23]:
# CREDIT: Dr. Vanderheyden wrote this code.
accuracies = []
for f in features:

    x = xtrain[f].values.reshape(-1, 1)
    y = y_train.reshape(-1, 1)
    ## LIN ##############################
    log_reg.fit(x, y)
    acc, auc = get_acc_auc(y, log_reg.predict(x))
    ## LOG #############################   
    xl = np.log(x - np.min(x) + 1)
    log_reg.fit(xl, y)
    lcc, luc = get_acc_auc(y, log_reg.predict(xl))
    if lcc / acc >= 1.1 or luc / auc >= 1.05:  # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + " log"] = xl
        xvalid[f + " log"] = np.log(xvalid[f].values.reshape(-1, 1) - np.min(xtrain[f])+1)
        xtest[f + " log"] = np.log(xtest[f].values.reshape(-1, 1) - np.min(xtrain[f])+1)
    ## EXP #############################   
    xe = np.exp(x)
    log_reg.fit(xe, y)
    ecc, euc = get_acc_auc(y, log_reg.predict(xe))
    if ecc / acc >= 1.1 or euc / auc >= 1.05: 
        xtrain[f + " exp"] = xe
        xvalid[f + " exp"] = np.exp(xvalid[f].values.reshape(-1, 1))
        xtest[f + " exp"] = np.exp(xtest[f].values.reshape(-1, 1))
    ## POLY ############################# 
    poly.fit(x)
    xp = poly.transform(x)
    log_reg.fit(xp, y)
    pcc, puc = get_acc_auc(y, log_reg.predict(xp))
    if pcc / acc >= 1.1 or puc / auc >= 1.05:  # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + " p2"] = x**2
        xtrain[f + " p3"] = x**3
        xvalid[f + " p2"] = (xvalid[f].values)**2
        xvalid[f + " p3"] = (xvalid[f].values)**3
        xtest[f + " p2"] = (xtest[f].values)**2
        xtest[f + " p3"] = (xtest[f].values)**3
    ## BIN #############################
    xmin = x.min()
    rnge = x.max() - xmin
    xtrn = 0 + ((x - xmin) > 1 * rnge / 10) + ((x - xmin) > 2 * rnge / 10) + ((x - xmin) > 3 * rnge / 10) + ( # the objects in each
                (x - xmin) > 4 * rnge / 10) + ((x - xmin) > 5 * rnge / 10) + ((x - xmin) > 6 * rnge / 10) + ( # bracket returns true
                (x - xmin) > 7 * rnge / 10) + ((x - xmin) > 8 * rnge / 10) + ((x - xmin) > 9 * rnge / 10)     # or false 
    xval = 0 + ((xvalid[f] - xmin) > 1 * rnge / 10) + ((xvalid[f] - xmin) > 2 * rnge / 10) + ((xvalid[f] - xmin) > 3 * rnge / 10) + (
                (xvalid[f] - xmin) > 4 * rnge / 10) + ((xvalid[f] - xmin) > 5 * rnge / 10) + ((xvalid[f] - xmin) > 6 * rnge / 10) + (
                (xvalid[f] - xmin) > 7 * rnge / 10) + ((xvalid[f] - xmin) > 8 * rnge / 10) + ((xvalid[f] - xmin) > 9 * rnge / 10)
    xtst = 0 + ((xtest[f] - xmin) > 1 * rnge / 10) + ((xtest[f] - xmin) > 2 * rnge / 10) + ((xtest[f] - xmin) > 5 * rnge / 10) + (
                (xtest[f] - xmin) > 3 * rnge / 10) + ((xtest[f] - xmin) > 4 * rnge / 10) + ((xtest[f] - xmin) > 6 * rnge / 10) + (
                (xtest[f] - xmin) > 7 * rnge / 10) + ((xtest[f] - xmin) > 8 * rnge / 10) + ((xtest[f] - xmin) > 9 * rnge / 10)
    encoder = TargetEncoder()
    encoder.fit(xtrn, y)
    xb = encoder.transform(xtrn)
    log_reg.fit(xb, y)
    bcc, buc = get_acc_auc(y, log_reg.predict(xb))
    if bcc / acc >= 1.1 or buc / auc >= 1.05: # if bin accuracy is 110% of linear accuracy or ... AUC is 105% ...
        xtrain[f + " Bin"] = xb
        xvalid[f + " Bin"] = encoder.transform(xval)
        xtest[f + " Bin"] = encoder.transform(xtst)
    ## COMPLETION #############################
    lDa = lcc / acc
    eDa = ecc / acc
    pDa = pcc / acc
    bDa = bcc / acc
    lda = luc / auc
    eda = euc / auc
    pda = puc / auc
    bda = buc / auc
    accuracies.append([f, acc, lcc, ecc, pcc, bcc, auc, luc, euc, puc, buc, lDa, eDa, pDa, bDa, lda, eda, pda, bda])
###############################################

colums = ["Feature","ACC: Linear", "ACC: Log", "ACC: Exp", "ACC: Poly3","ACC: Bin",
                    "AUC: Simple Linear", "AUC: Log", "AUC: Exp","AUC: Poly3", "AUC: Bin",
                    "ACC: LOG / Linear", "ACC: EXP / Linear", "ACC: Poly3 / Linear", "ACC: Bin / Linear",
                    "AUC: LOG / Linear", "AUC: EXP / Linear", "AUC: Poly3 / Linear", "AUC: Bin / Linear"]
accDf = pd.DataFrame(accuracies, columns = colums)
accDf

C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Nate\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y t

,Feature,ACC: Linear,ACC: Log,ACC: Exp,ACC: Poly3,ACC: Bin,AUC: Simple Linear,AUC: Log,AUC: Exp,AUC: Poly3,AUC: Bin,ACC: LOG / Linear,ACC: EXP / Linear,ACC: Poly3 / Linear,ACC: Bin / Linear,AUC: LOG / Linear,AUC: EXP / Linear,AUC: Poly3 / Linear,AUC: Bin / Linear
0,ST_FIPS,1953.006187,1986.644238,1896.602990,1994.798917,1953.006187,0.505308,0.511244,0.505415,0.513857,0.505308,1.017224,0.971120,1.021399,1.000000,1.011748,1.000212,1.016920,1.000000
1,CITY Encoded,2013.486723,1997.517144,2093.844290,2012.467389,2093.844290,0.667642,0.668695,0.646575,0.667381,0.646575,0.992069,1.039910,0.999494,1.039910,1.001578,0.968447,0.999610,0.968447
2,INSTNM Encoded,2254.559423,2254.559423,1615.946120,2254.559423,2254.559423,0.518965,0.518965,0.507182,0.518965,0.518965,1.000000,0.716746,1.000000,1.000000,1.000000,0.977295,1.000000,1.000000
3,FIPS_1,2259.316319,2259.316319,1610.000000,2259.316319,2259.316319,0.504960,0.504960,0.500000,0.504960,0.504960,1.000000,0.712605,1.000000,1.000000,1.000000,0.990178,1.000000,1.000000
4,FIPS_2,1611.529002,1611.529002,2269.000000,1611.529002,1611.529002,0.500390,0.500390,0.500000,0.500390,0.500390,1.000000,1.407980,1.000000,1.000000,1.000000,0.999220,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,num1PC04,1923.445476,1897.112658,2269.000000,1922.596030,2066.322248,0.492449,0.482184,0.500000,0.490285,0.524337,0.986310,1.179654,0.999558,1.074282,0.979156,1.015334,0.995606,1.064754
233,num2PC01,1761.201341,1778.699923,2269.000000,1731.640629,1617.984790,0.620200,0.628562,0.500000,0.603094,0.508233,1.009936,1.288325,0.983216,0.918682,1.013483,0.806192,0.972418,0.819467
234,num2PC02,1811.828306,1787.873937,2269.000000,1913.591905,1623.421243,0.553138,0.542773,0.500000,0.621424,0.511568,0.986779,1.252326,1.056166,0.896013,0.981261,0.903934,1.123452,0.924848
235,num5PC01,1843.087909,1862.115494,1786.684713,1838.161124,1814.886311,0.602179,0.592880,0.609188,0.620211,0.609665,1.010324,0.969397,0.997327,0.984699,0.984559,1.011639,1.029944,1.012432


In [10]:
print(xtrain.shape)
accDf.to_csv("acc_df.csv")
accDf.loc[:, ["ACC: LOG / Linear", "ACC: EXP / Linear", "ACC: Poly3 / Linear", "ACC: Bin / Linear",
              "AUC: LOG / Linear", "AUC: EXP / Linear", "AUC: Poly3 / Linear", "AUC: Bin / Linear"]
             ].sort_values("ACC: LOG / Linear", ascending = True).head(5)

(3879, 460)


,ACC: LOG / Linear,ACC: EXP / Linear,ACC: Poly3 / Linear,ACC: Bin / Linear,AUC: LOG / Linear,AUC: EXP / Linear,AUC: Poly3 / Linear,AUC: Bin / Linear
38,0.709564,0.986298,0.986298,1.000000,1.000000,1.000048,1.000048,1.000000
82,0.709670,0.709670,1.000000,1.000000,0.999820,0.999820,1.000000,1.000000
64,0.712634,0.709564,0.998503,0.996930,1.003204,1.000000,1.001804,0.996796
172,0.878370,0.862692,0.886210,1.017741,1.084874,1.065590,1.202026,1.032295
171,0.880776,0.778138,1.003531,1.036211,0.967333,0.839179,0.992735,0.971521


In [ ]:
# drp = ["ST_FIPS", "MENONLY", "WOMENONLY", "TRIBAL", "PBI", "NANTI", "ANNHI", "ANNHI_0.0", "HBCU", 
#        "PBI_0.0", "TRIBAL_0.0", "HSI", "HBCU_0.0", "ANNHI", "HBCU", "HSI", "NANTI_0.0", "HSI_0.0",
#        "MAIN_0", "MAIN", "HCM2", "AANAPII", "AANAPII_0.0", "CURROPER", "CURROPER_0", "ZIP2", "ZIP4",
#        "ST_FIPS","ACCRED_NONE","OPENADMP","HIGHDEG","PREDDEG","ICLEVEL","CONTROL","OPEFLAG","BBRR2_FED_UG_N",
#        "ZIP5","BBRR2_FED_UGNOCOMP_N","LPSTAFFORD_CNT", "BBRR2_PP_UG_N","LPPPLUS_CNT"]

# xtrain = xtrain.drop(drp, axis = 1)
# xvalid = xvalid.drop(drp, axis = 1)
# xtest = xtest.drop(drp, axis = 1)

# dropped_due_2_modelling_iss = ["ACCRED_NASD exp", "ACCRED_ACAOM exp", "ST_FIPS","MENONLY",
#                                "WOMENONLY","HBCU","PBI","ANNHI","TRIBAL","AANAPII","HSI","NANTI",
#                                "ACCRED_DETC exp","ACCRED_DETC exp","ACCRED_NASAD exp",
#                                "ACCRED_WASCCS|WASCCS exp","OPEFLAG_2 exp","ACCRED_ABFSE exp",
#                                "ACCRED_JRCERT exp","ACCRED_MEAC exp","ACCRED_NAST exp",
#                                "ACCRED_NLNAC exp","FIPS_2 exp","ACCRED_OKSBVT exp",
#                                "ACCRED_WASCCS|WASCCS Bin","FIPS_56 exp","HIGHDEG_3 log",
#                                "NANTI_1.0 exp","HSI_1.0","HBCU_1.0","ANNHI_1.0",
#                                "NANTI_1.0 exp","PBI_1.0","NANTI exp","AANAPII",
#                                "ACCRED_ABFSE exp","ACCRED_ACAOM exp","ACCRED_ATSUSC exp",
#                                "ACCRED_DETC exp","ACCRED_JRCERT exp","ACCRED_MEAC exp",
#                                "ACCRED_NASAD exp","ACCRED_NASD exp","ACCRED_NASM exp",
#                                "ACCRED_NAST exp","ACCRED_NLNAC exp","ACCRED_NYBRE exp",
#                                "ACCRED_NYBRE p2","ACCRED_NYBRE p3","ACCRED_OKSBVT exp",
#                                "ACCRED_WASCCS|WASCCS Bin","ACCRED_WASCCS|WASCCS exp",
#                                "ACCRED_WASCJC exp","ANNHI","ANNHI exp","ANNHI_1.0 exp",
#                                "BBRR2_FED_UG_N exp","BBRR2_FED_UGCOMP_N exp",
#                                "BBRR2_FED_UGNOCOMP_N exp","CURROPER","DISTANCEONLY exp",
#                                "FIPS_10 exp","FIPS_11 exp","FIPS_15 exp","FIPS_19 exp",
#                                "FIPS_2 exp","FIPS_20 exp","FIPS_23 exp","FIPS_24 exp",
#                                "FIPS_27 exp","FIPS_30 exp","FIPS_31 exp","FIPS_33 exp",
#                                "FIPS_35 exp","FIPS_38 exp","FIPS_40 exp","FIPS_41 exp",
#                                "FIPS_44 exp","FIPS_46 exp","FIPS_50 exp","FIPS_53 exp",
#                                "FIPS_55 exp","FIPS_56 exp","FIPS_8 exp","FIPS_9 exp",
#                                "HBCU","HCM2","HIGHDEG_3 log","HSI","LPSTAFFORD_AMT exp",
#                                "LPSTAFFORD_CNT exp","MAIN","MENONLY exp","NANTI",
#                                "NANTI exp","NANTI_1.0 exp","num2PC02 exp","OPEFLAG_2 exp",
#                                "OPEFLAG_3 exp","PBI","TRIBAL","TRIBAL_0.0","NANTI_0.0",
#                                "MAIN_0","HCM2_0","CURROPER_0","AANAPII_0.0","ZIP4","ZIP4 p3",
#                                "ZIP4 p2","ZIP4 p3","ZIP4 p2","HIGHDEG","PREDDEG","ICLEVEL",
#                                "BBRR2_FED_UG_N","BBRR2_FED_UGCOMP_N","BBRR2_FED_UGNOCOMP_N",
#                                "ZIP2 p2","LPPPLUS_CNT exp","BBRR2_FED_UGNOCOMP_N p3","ICLEVEL p3",
#                                "BBRR2_FED_UGCOMP_NOPROG log","pcipPC06 Bin","cipPC31 log",
#                                "ICLEVEL exp","cipPC26 log","cipPC11 log","cipPC36 log",
#                                "cipPC33 log","cipPC32 log","cipPC24 log","cipPC10 log",
#                                "cipPC19 log","geoPC04 Bin","cipPC37 log","cipPC14 log",
#                                "pcipPC05 Bin","LPPPLUS_AMT exp"]

# xtrain = xtrain.drop(dropped_due_2_modelling_iss, axis = 1)
# xvalid = xvalid.drop(dropped_due_2_modelling_iss, axis = 1)
# xtest = xtest.drop(dropped_due_2_modelling_iss, axis = 1)


### Checking the Correlation of features

In [11]:
corrs = []
contFeat = list(xtrain.columns)
contFeat_length = len(contFeat)

for i in range(contFeat_length):
    for j in range(i + 1, contFeat_length):
        feati = xtrain[contFeat[i]].values.flatten()
        featj = xtrain[contFeat[j]].values.flatten()

        corr, _ = pearsonr(feati, featj)
        corrs.append([corr, abs(corr), contFeat[i], contFeat[j]])


In [13]:
correl = pd.DataFrame(corrs, columns = ["P_Corr", "P_Corr_abs", "feat1", "feat2"])
################################
tot_var_pair = (contFeat_length * (contFeat_length - 1) / 2)
num_var_g50 = len(correl[abs(correl["P_Corr"]) > 0.5])
pct_var_g50 = np.round(num_var_g50 / tot_var_pair * 100, 4)
print(xtrain.shape)
print(int(tot_var_pair), " : Total number of features pairs:")
print(num_var_g50, "   : Number of features pairs with absolute Pearson Correl above 0.5:")
print(pct_var_g50, "% : Percent of features pairs with absolute Pearson Correl above 0.5:")

(3879, 460)
105570  : Total number of features pairs:
764    : Number of features pairs with absolute Pearson Correl above 0.5:
0.7237 % : Percent of features pairs with absolute Pearson Correl above 0.5:


In [14]:
correl[correl["P_Corr_abs"] > .6].sort_values("P_Corr_abs", ascending=False).iloc[1:40]#.to_csv("corr.csv")

correl.to_csv("corr.csv")

# Modelling

#### Logistic Regression

In [ ]:
from sklearn.metrics import roc_auc_score

ccc = [0.0001]
n_est = [1000]

logReg_lst = []

for cc in ccc:
    for nest in n_est:
        logReg = LogisticRegression(solver = "saga",
                                    random_state = rand_state, 
                                    penalty = "l1", 
                                    C = cc, 
                                    class_weight = "balanced",
                                    max_iter = nest)
        logReg.fit(xtrain, y_train)

        Tpred = logReg.predict(xtrain)
        vpred = logReg.predict(xvalid)
        tpred = logReg.predict(xtest)

        Tprob = logReg.predict_proba(xtrain)
        vprob = logReg.predict_proba(xvalid)
        tprob = logReg.predict_proba(xtest)

        logReg_lst.append(["saga", "l1", cc, nest, np.sum(logReg.coef_==0),
                        round(roc_auc_score(y_train, Tprob[:,1]), 4), round(roc_auc_score(y_test, tprob[:,1]), 4), round(roc_auc_score(y_valid, vprob[:,1]), 4),
                        round(1 - np.mean(abs(y_train - Tpred)), 4), round(1 - np.mean(abs(y_test - tpred)), 4), round(1 - np.mean(abs(y_valid- vpred)), 4),
                        ])

colums = ["solver", "penalty", "C", "max_iter", "coef_eq_zero",
          "Train_AUC", "Test_AUC", "Valid_AUC",
          "Train_Accurcy", "Test_Accurcy", "Valid_Accurcy",
          ]
logReg_outcomes = pd.DataFrame(logReg_lst, columns = colums)

In [ ]:
df = logReg_outcomes
df

In [ ]:
import dalex as dx

from dalex._explainer.yhat import yhat_proba_default

expReduced = dx.Explainer(logReg, xtest, y_test.reshape(-1,))
explanationReduced = expReduced.model_parts()
explanationReduced.plot(max_vars=15)

In [ ]:
logReg_lst = []
lll = [0.1, 0.5, 0.9]
ccc = [100, 1, 0.01]
n_est = [10, 50, 100]

for cc in ccc:
    logReg = LogisticRegression(solver = "saga",
                                random_state = rand_state,
                                penalty = "l1",
                                C = cc,
                                class_weight = 'balanced',
                                max_iter = 100)
    logReg.fit(xtrain, y_train)

    Tpred = logReg.predict(xtrain)
    vpred = logReg.predict(xvalid)
    tpred = logReg.predict(xtest)

    Tprob = logReg.predict_proba(xtrain)
    vprob = logReg.predict_proba(xvalid)
    tprob = logReg.predict_proba(xtest)

    logReg_lst.append(["saga", "l1", cc, None, 100, np.sum(logReg.coef_==0),
                        round(roc_auc_score(y_train, Tprob[:,1]), 4), round(roc_auc_score(y_test, tprob[:,1]), 4), round(roc_auc_score(y_valid, vprob[:,1]), 4),
                        round(1 - np.mean(abs(y_train - Tpred)), 4), round(1 - np.mean(abs(y_test - tpred)), 4), round(1 - np.mean(abs(y_valid- vpred)), 4),
                        ])

for cc in ccc:
    logReg= LogisticRegression(solver = "saga",
                                random_state = rand_state,
                                penalty = "l2",
                                C = cc,
                                max_iter = 100)

    logReg.fit(xtrain, y_train)

    Tpred = logReg.predict(xtrain)
    vpred = logReg.predict(xvalid)
    tpred = logReg.predict(xtest)

    Tprob = logReg.predict_proba(xtrain)
    vprob = logReg.predict_proba(xvalid)
    tprob = logReg.predict_proba(xtest)

    logReg_lst.append(["saga", "l2", cc, None, 100, np.sum(logReg.coef_==0),
                        round(roc_auc_score(y_train, Tprob[:,1]), 4), round(roc_auc_score(y_test, tprob[:,1]), 4), round(roc_auc_score(y_valid, vprob[:,1]), 4),
                        round(1 - np.mean(abs(y_train - Tpred)), 4), round(1 - np.mean(abs(y_test - tpred)), 4), round(1 - np.mean(abs(y_valid- vpred)), 4),
                        ])

for cc in ccc:
    for ll in lll:
        logReg = LogisticRegression(solver = "saga",
                                    random_state = rand_state,
                                    penalty = "elasticnet",
                                    C = cc,
                                    l1_ratio = ll,
                                    max_iter = 100)

        logReg.fit(xtrain, y_train)

        Tpred = logReg.predict(xtrain)
        vpred = logReg.predict(xvalid)
        tpred = logReg.predict(xtest)

        Tprob = logReg.predict_proba(xtrain)
        vprob = logReg.predict_proba(xvalid)
        tprob = logReg.predict_proba(xtest)

        logReg_lst.append(["saga", "elasticnet", cc, ll, 100, np.sum(logReg.coef_==0),
                        round(roc_auc_score(y_train, Tprob[:,1]), 4), round(roc_auc_score(y_test, tprob[:,1]), 4), round(roc_auc_score(y_valid, vprob[:,1]), 4),
                        round(1 - np.mean(abs(y_train - Tpred)), 4), round(1 - np.mean(abs(y_test - tpred)), 4), round(1 - np.mean(abs(y_valid- vpred)), 4),
                        ])

logReg_outcomes = pd.DataFrame(logReg_lst, columns = ["solver", "penalty", "C", "l1_ratio", "max_iter", "coef_eq_zero",
                                                      "Train_AUC", "Test_AUC", "Valid_AUC", "Train_Accurcy",
                                                      "Test_Accurcy", "Valid_Accurcy"])

# logReg_outcomes.to_csv("logReg_out.csv")

In [ ]:
logReg_outcomes.sort_values("Test_Accurcy", ascending = False)

#### Random Forest Classification

In [ ]:
randforest_lst = []
crit = ["gini", "entropy"]
depth = [5, 6, 7]
leaf = [1, 2, 3]
n_est = [1000]

for crits in crit:
        for depths in depth:
                for leafs in leaf:
                        for nest in n_est:
                                Random_forest = RandomForestClassifier(n_estimators = nest, #  The number of trees in the forest.
                                                                        criterion = crits,
                                                                        max_depth = depths,  
                                                                        min_samples_leaf = leafs,
                                                                        class_weight = 'balanced_subsample',
                                                                        random_state = rand_state)
                                Random_forest.fit(xtrain, y_train)

                                Tpred = Random_forest.predict(xtrain)
                                vpred = Random_forest.predict(xvalid)
                                tpred = Random_forest.predict(xtest)

                                Tprob = Random_forest.predict_proba(xtrain)
                                vprob = Random_forest.predict_proba(xvalid)
                                tprob = Random_forest.predict_proba(xtest)

                                randforest_lst.append([leafs, depths, crits, nest,
                                                       round(roc_auc_score(y_train, Tprob[:,1]), 4), round(1 - np.mean(abs(y_train - Tpred)), 4),
                                                       round(roc_auc_score(y_test, tprob[:,1]), 4), round(1 - np.mean(abs(y_test - tpred)), 4),
                                                       round(roc_auc_score(y_valid, vprob[:,1]), 4), round(1 - np.mean(abs(y_valid- vpred)), 4)])
colums = ["leaf", "depth", "criterion", "n_est", "Train_AUC", "Train_Accurcy", "Test_AUC", "Test_Accurcy", "Valid_AUC", "Valid_Accurcy"]
rand_forest_outcomes = pd.DataFrame(randforest_lst, columns = colums)
rand_forest_outcomes

In [ ]:
rand_forest_outcomes.sort_values("Test_Accurcy", ascending = False)

#### Neural Network

In [ ]:
x1 = xtrain[(y_train == 1).values]
y1 = y_train[(y_train == 1).values]
x, y = resample(x1, y1, 
                replace = True,
                n_samples = len(y_train) - 2*sum(y_train),
                random_state = rand_state)

xt = pd.concat([xtrain, x])
yt = pd.concat([y_train, y])

xs, ys = shuffle(xt, yt,
                 random_state = rand_state)

MLPClst = []

lr = [0.1]  # [0.1,0.3,0.5,0.7,0.9]
hl = [(16, 2),(32,16),(16,8),(8,4),(16,4)]


for learn in lr:
    for hidden in hl:
        mlpc = MLPClassifier(solver = 'adam', 
                             learning_rate_init = learn, 
                             alpha = 0.0001, 
                             batch_size = 16, 
                             hidden_layer_sizes = hidden, 
                             max_iter = 3000,
                             random_state = rand_state)

        mlpc.fit(xs, ys)

        Tpred = mlpc.predict(xtrain)
        vpred = mlpc.predict(xvalid)
        tpred = mlpc.predict(xtest)
        
        Tprob = mlpc.predict_proba(xtrain)
        vprob = mlpc.predict_proba(xvalid)
        tprob = mlpc.predict_proba(xtest)

        MLPClst.append([learn, hidden,
                roc_auc_score(y_train, mlpc.predict_proba(xtrain)[:,1]), mlpc.score(xtrain, y_train),
                roc_auc_score(y_test, mlpc.predict_proba(xtest)[:,1]), mlpc.score(xtest, y_test),
                roc_auc_score(y_valid, mlpc.predict_proba(xvalid)[:,1]), mlpc.score(xvalid, y_valid),
                    ])
colums = ["learning_rate", "hidden_nodes", "Train_AUC", "Train_Accurcy", "Test_AUC", "Test_Accurcy", "Valid_AUC", "Valid_Accurcy"]
MLP_outcomes = pd.DataFrame(MLPClst, columns = colums)
# MLP_outcomes.to_csv("MLP_out.csv")

#### Support Vector Machine

In [ ]:
rbf_gamma = [0.001] # np.array(range(100,1000000,100)) / 1000000
poly_degree = [2] # np.array(range(2, 11, 1))

coefficients = [-1, 0, 1] # 
error_penalty = [1,10,50] # [1, 2, 3, 4, 5]

svc_lst = []

for penalty in error_penalty:
    lin = svm.SVC(kernel = 'linear', 
                  C = penalty,
                  class_weight = "balanced",
                  random_state = rand_state)
                
    lin.fit(xtrain, y_train)

    Tpred = lin.predict(xtrain)
    vpred = lin.predict(xvalid)
    tpred = lin.predict(xtest)

    Tprob = lin.decision_function(xtrain)
    vprob = lin.decision_function(xvalid)
    tprob = lin.decision_function(xtest)

    svc_lst.append(["linear", penalty, None, None, None,
                    round(roc_auc_score(y_train, Tprob), 4), round(roc_auc_score(y_test, tprob), 4), round(roc_auc_score(y_valid, vprob), 4),
                    round(1 - np.mean(abs(y_train - Tpred)), 4), round(1 - np.mean(abs(y_test - tpred)), 4), round(1 - np.mean(abs(y_valid- vpred)), 4),
                    ])

    for coef in coefficients:
        for g in rbf_gamma:
            rbf = svm.SVC(kernel = "rbf",
                          gamma = g, 
                          C = penalty,
                          coef0 = coef,
                          class_weight = "balanced",
                          random_state = rand_state)
            
            rbf.fit(xtrain, y_train)
            
            Tpred = rbf.predict(xtrain)
            vpred = rbf.predict(xvalid)
            tpred = rbf.predict(xtest)

            Tprob = rbf.decision_function(xtrain)
            vprob = rbf.decision_function(xvalid)
            tprob = rbf.decision_function(xtest)
         
            svc_lst.append(["rbf", penalty, coef, g, None,
                            round(roc_auc_score(y_train, Tprob), 4), round(roc_auc_score(y_test, tprob), 4), round(roc_auc_score(y_valid, vprob), 4),
                            round(1 - np.mean(abs(y_train - Tpred)), 4), round(1 - np.mean(abs(y_test - tpred)), 4), round(1 - np.mean(abs(y_valid- vpred)), 4),
                            ])
    
        for degr in poly_degree:
            poly = svm.SVC(kernel = "poly",
                           degree = degr,
                           C = penalty,
                           coef0 = coef,
                           class_weight = "balanced",
                           random_state = rand_state)

            poly.fit(xtrain, y_train)
            
            Tpred = poly.predict(xtrain)
            vpred = poly.predict(xvalid)
            tpred = poly.predict(xtest)

            Tprob = poly.decision_function(xtrain)
            vprob = poly.decision_function(xvalid)
            tprob = poly.decision_function(xtest)
         
            svc_lst.append(["poly", penalty, coef, None, degr,
                            round(roc_auc_score(y_train, Tprob), 4), round(roc_auc_score(y_test, tprob), 4), round(roc_auc_score(y_valid, vprob), 4),
                            round(1 - np.mean(abs(y_train - Tpred)), 4), round(1 - np.mean(abs(y_test - tpred)), 4), round(1 - np.mean(abs(y_valid- vpred)), 4),
                            ])
                           
colums = ["kernel", "penalty", "coef", "gamma", "degree", "Train_AUC",  "Test_AUC", "Valid_AUC", "Train_Accurcy", "Test_Accurcy", "Valid_Accurcy"]
svc_outcome = pd.DataFrame(svc_lst, columns = colums)

In [ ]:
svc_outcome

#### XGBoost

In [ ]:
xgblst = []

x1 = xtrain[(y_train == 1).values]
y1 = y_train[(y_train == 1).values]

x, y = resample(x1, y1,
                replace = True,
                n_samples = (len(y_train) - 2*sum(y_train)),
                random_state = rand_state)

xt = pd.concat([xtrain, x])
yt = pd.concat([y_train, y])

xs, ys = shuffle(xt, yt, 
                 random_state = rand_state)

lr = np.append([.001, .005], np.array(range(0, 105, 5)) / 100)
n_est = np.array(range(10, 501, 10))

for learn in lr:
    for nest in n_est:
        xgb = XGBClassifier(learning_rate = learn, 
                            n_estimators = 300)
        
        eval_set = [(xtrain, y_train), (xvalid, y_valid)]
        
        xgb.fit(xs, ys, 
                eval_metric='auc', 
                eval_set = eval_set)

        Tpred = xgb.predict(xtrain)
        vpred = xgb.predict(xvalid)
        tpred = xgb.predict(xtest)

        xgblst.append([learn, nest,
                       roc_auc_score(y_train, xgb.predict_proba(xtrain)[:,1]), xgb.score(xtrain, Tpred),
                       roc_auc_score(y_test, xgb.predict_proba(xtest)[:,1]), xgb.score(xtest, tpred),
                       roc_auc_score(y_valid, xgb.predict_proba(xvalid)[:,1]), xgb.score(xvalid, vpred),
                     ])

colums = ["learn_rate", "n_estimator", "Train_AUC", "Train_Accurcy", "Test_AUC", "Test_Accurcy", "Valid_AUC", "Valid_Accurcy"]
xgboost_outcome = pd.DataFrame(xgblst, columns = colums)
xgboost_outcome.to_csv("xgboost_out.csv")

#### 